In [1]:
import tensorflow as tf

saved_model_dir = "c:/Users/jbetk/Documents/data/ml/saved_models/sentiment_mse_distilbert_yelp_amazon/"

converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.experimental_new_converter = True

tflite_model = converter.convert()
open(saved_model_dir + "converted_model.tflite", "wb").write(tflite_model)

66892912

In [4]:
interpreter = tf.lite.Interpreter(model_path=saved_model_dir + "/converted_model.tflite")
print(interpreter.get_input_details())

ValueError: Input array not provided for operation 'reshape'.


In [15]:
import tensorflow as tf
from transformers import TFRobertaModel

model = TFRobertaModel.from_pretrained('roberta-base')

input_spec = [tf.TensorSpec([1, 128], tf.int32), tf.TensorSpec([1, 128], tf.int32)]
model._set_inputs(input_spec, training=False)

print(model.inputs)
print(model.outputs)

converter = tf.lite.TFLiteConverter.from_keras_model(model)

# For normal conversion:
#converter.target_spec.supported_ops = [tf.lite.OpsSet.SELECT_TF_OPS]

# For conversion with FP16 quantization:
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
# converter.target_spec.supported_types = [tf.float16]
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.experimental_new_converter = True

# For conversion with hybrid quantization:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.experimental_new_converter = True

tflite_model = converter.convert()

open("distilbert-squad-384.tflite", "wb").write(tflite_model)

[<tf.Tensor 'input_1_11:0' shape=(None, 128) dtype=int32>, <tf.Tensor 'input_2_9:0' shape=(None, 128) dtype=int32>]
[<tf.Tensor 'tf_roberta_model_1/Identity:0' shape=(None, 128, 768) dtype=float32>, <tf.Tensor 'tf_roberta_model_1/Identity_1:0' shape=(None, 768) dtype=float32>]


ConverterError: See console for info.
2020-02-03 14:16:20.869205: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library cudart64_100.dll
2020-02-03 14:16:25.853657: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Cumsum
2020-02-03 14:16:25.854123: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Erf
2020-02-03 14:16:25.854715: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Erf
2020-02-03 14:16:25.855259: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Erf
2020-02-03 14:16:25.855869: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Erf
2020-02-03 14:16:25.856324: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Erf
2020-02-03 14:16:25.856863: I tensorflow/lite/toco/import_tensorflow.cc:659] Converting unsupported operation: Erf
2020-02-0